In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot
import seaborn as sns
import tensorflow as tf
import keras
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
import pickle
import os
import NN

In [29]:
import dataset_utils
import importlib

In [30]:
importlib.reload(dataset_utils)

<module 'dataset_utils' from 'c:\\Users\\cyber\\Desktop\\University\\P4P\\github\\P4P-mmWave\\MMWave_Radar_Human_Tracking_and_Fall_detection\\dataset_utils.py'>

In [31]:
DATA_DIR = "3_class_data"
NUM_POINTS = 100
NUM_CLASSES = 2
BATCH_SIZE = 32
train_points, test_points, train_labels, test_labels, CLASS_MAP = dataset_utils.parse_dataset(NUM_POINTS, DATA_DIR)

processing class: jumping
processing class: speedwalking
processing class: walking


In [32]:
print(test_labels)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


In [33]:
print(train_points)

[[[ 2.3877e-01  2.5122e-01  5.2765e-02 -7.5146e-01  2.2700e+02]
  [ 3.5474e-01  2.5122e-01  1.5710e-01  5.0098e-01  2.0100e+02]
  [ 3.5474e-01  2.3584e-01  1.5332e-01  5.0098e-01  2.0100e+02]
  ...
  [ 7.0654e-01  6.2305e-01  3.2593e-01 -5.0098e-01  2.1300e+02]
  [ 0.0000e+00  7.9688e-01  1.0000e+00 -5.0098e-01  3.0600e+02]
  [ 5.0635e-01  6.2305e-01  4.8828e-01  2.5049e-01  2.8300e+02]]

 [[ 6.9141e-01  9.5215e-01  4.6216e-01 -7.5146e-01  2.1200e+02]
  [ 0.0000e+00  1.0000e+00  3.1152e-01  3.7573e-01  1.8800e+02]
  [ 5.8350e-01  7.7271e-02  7.9980e-01 -5.0098e-01  3.3300e+02]
  ...
  [ 7.5049e-01  3.9917e-01  7.9834e-01  1.2524e-01  1.4800e+02]
  [ 6.2695e-01  5.2393e-01  5.5762e-01 -7.5146e-01  2.9600e+02]
  [ 2.8271e-01  3.8257e-01  6.0425e-02  3.7573e-01  2.0200e+02]]

 [[ 2.8271e-01  3.8257e-01  6.0425e-02  3.7573e-01  2.0200e+02]
  [ 5.4102e-01  2.1191e-01  6.6162e-01 -3.7573e-01  2.5300e+02]
  [ 5.5029e-01  2.5342e-01  1.0000e+00 -5.0098e-01  2.1100e+02]
  ...
  [ 1.0000e+00  8.

In [34]:
from tensorflow import data as tf_data
import keras
#import PointNET

keras.utils.set_random_seed(seed=42)

In [35]:
train_size = 0.8
dataset = tf_data.Dataset.from_tensor_slices((train_points, train_labels))
test_dataset = tf_data.Dataset.from_tensor_slices((test_points, test_labels))
train_dataset_size = int(len(dataset) * train_size)

In [36]:
dataset = dataset.shuffle(len(train_points)).map(NN.augment)
test_dataset = test_dataset.shuffle(len(test_points)).batch(BATCH_SIZE)
train_dataset = dataset.take(train_dataset_size).batch(BATCH_SIZE)
validation_dataset = dataset.skip(train_dataset_size).batch(BATCH_SIZE)

In [37]:
inputs = keras.Input(shape=(NUM_POINTS, 5))

In [38]:
import matplotlib
import numpy as np
from matplotlib import pyplot as plt
from matplotlib.ticker import LinearLocator
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Conv3D, MaxPooling3D, Embedding, LSTM, Bidirectional, Reshape, Concatenate

In [40]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization

# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

#model.add(layers.Conv1D(filters=128, kernel_size=3, activation='relu', input_shape=input_shape))
#model.add(layers.MaxPooling1D(pool_size=2))
#model.add(Dropout(0.25))
#model.add(BatchNormalization())

# # 2. LSTM layer
# model.add(layers.LSTM(50, return_sequences=True))
# model.add(layers.LSTM(50))

# 3. Dense layers
#model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(layers.Dense(1, activation='linear'))  # Output layer (you can adjust the output size and activation)
model.add(layers.Dense(1, activation='softmax'))


In [41]:
# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Summary of the model
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_4 (Conv1D)           (None, None, 64)          1024      
                                                                 
 max_pooling1d_4 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, None, 64)          0         
                                                                 
 batch_normalization_4 (Bat  (None, None, 64)          256       
 chNormalization)                                                
                                                                 
 dense_4 (Dense)             (None, None, 64)          4160      
                                                                 
 dense_5 (Dense)             (None, None, 1)          

In [42]:
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=3)
# training
history = model.fit(train_points, train_labels, epochs=30, batch_size=16, validation_data=(test_points, test_labels), verbose=1, callbacks=[early_stopping])

Epoch 1/30
15/15 [==============================] - 2s 24ms/step - loss: 0.6667 - mae: 0.6667 - val_loss: 0.6667 - val_mae: 0.6667
Epoch 2/30
15/15 [==============================] - 0s 7ms/step - loss: 0.6667 - mae: 0.6667 - val_loss: 0.6667 - val_mae: 0.6667
Epoch 3/30
15/15 [==============================] - 0s 7ms/step - loss: 0.6667 - mae: 0.6667 - val_loss: 0.6667 - val_mae: 0.6667
Epoch 4/30
15/15 [==============================] - 0s 7ms/step - loss: 0.6667 - mae: 0.6667 - val_loss: 0.6667 - val_mae: 0.6667


In [43]:
test_loss, test_acc = model.evaluate(test_points, test_labels, verbose=1)

2/2 [==============================] - 0s 8ms/step - loss: 0.6667 - mae: 0.6667


In [44]:
print(test_acc)

0.6666668057441711


In [45]:
# pickle.dump(model, open("model.pkl", "wb"))

In [59]:
# Define the input shape (5 features)
input_shape = (None, 5)

# Define the model
model = models.Sequential()

# 1. CNN layer
model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu', input_shape=input_shape))
model.add(layers.MaxPooling1D(pool_size=2))
model.add(Dropout(0.25))
model.add(BatchNormalization())

# Flatten the output
model.add(layers.Flatten())

# Summary of the CNN model
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_8 (Conv1D)           (None, None, 64)          1024      
                                                                 
 max_pooling1d_8 (MaxPoolin  (None, None, 64)          0         
 g1D)                                                            
                                                                 
 dropout_8 (Dropout)         (None, None, 64)          0         
                                                                 
 batch_normalization_8 (Bat  (None, None, 64)          256       
 chNormalization)                                                
                                                                 
 conv1d_9 (Conv1D)           (None, None, 64)          12352     
                                                                 
 max_pooling1d_9 (MaxPoolin  (None, None, 64)         

In [60]:
import numpy as np

# Assuming sequence_length = 100 for this example
sequence_length = 100
#X = np.random.random((10, sequence_length, 5))  # Example data

# Extract features using CNN
X_Train_Features = model.predict(train_points)
X_Test_Features = model.predict(test_points)

2/2 [==============================] - 0s 0s/step


In [61]:
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

svm_classifer = svm.SVC(kernel='linear')
svm_classifer.fit(X_Train_Features, train_labels)

y_pred = svm_classifer.predict(X_Test_Features)

accuracy = accuracy_score(test_labels, y_pred)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 51.67


In [62]:
print(len(X_Train_Features))
print(len(X_Test_Features))
print(len(train_labels))
print(len(test_labels))

240
60
240
60


In [63]:
from sklearn.model_selection import GridSearchCV
param_grid={'C':[0.1,1,10,100],'gamma':[0.0001,0.001,0.1,1],'kernel':['rbf','poly']}
svc=svm.SVC(probability=True)
print("The training of the model is started, please wait for while as it may take few minutes to complete")
model=GridSearchCV(svc,param_grid, refit = True, verbose = 4, n_jobs=-1)
model.fit(X_Train_Features,train_labels)
print('The Model is trained well with the given images')
model.best_params_

pickle.dump(model,open('SVM_model.p', 'wb'))
y_pred=model.predict(X_Test_Features)

print(f"The models is {accuracy_score(y_pred,test_labels)*100}% accurate")

The training of the model is started, please wait for while as it may take few minutes to complete
Fitting 5 folds for each of 32 candidates, totalling 160 fits
The Model is trained well with the given images
The models is 56.666666666666664% accurate


In [64]:
with open('SVM_model.p', 'rb') as f:
    model = pickle.load(f)
y_pred = model.predict(X_Test_Features)
accuracy = accuracy_score(test_labels, y_pred)

In [65]:
print(accuracy)

0.5666666666666667
